In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random

In [2]:
# import our chat-bot intents file
import json

with open('intents.json') as json_data:
    intents = json.load(json_data)

words = []
classes = []
documents = []
ignore_words = ['?','!']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

62 documents
15 classes ['check_ahu', 'check_all_hvac', 'check_boiler_plant', 'check_chiller_plant', 'check_energy', 'check_future_weather', 'check_how_it_should_all_work', 'check_power', 'check_runtime', 'check_zone_temps', 'goodbye', 'greeting', 'options', 'thanks', 'what_to_do']
152 unique stemmed words ["'s", ',', 'a', 'ahu', 'air', 'al', 'alarm', 'algorithm', 'an', 'and', 'any', 'anyon', 'ar', 'area', 'at', 'awesom', 'bad', 'be', 'been', 'big', 'boil', 'box', 'build', 'bye', 'cal', 'can', 'cent', 'chat', 'chil', 'cold', 'consum', 'control', 'cool', 'could', 'damp', 'day', 'did', 'discharg', 'do', 'doe', 'doing', 'down', 'driv', 'duc', 'dud', 'econom', 'elect', 'energy', 'equip', 'fan', 'fir', 'for', 'fre', 'freez', 'frequ', 'from', 'front', 'fuel', 'get', 'going', 'good', 'goodby', 'handl', 'has', 'hello', 'help', 'hi', 'hot', 'hour', 'how', 'hvac', 'i', 'ilc', 'in', 'intellig', 'is', 'kw', 'kwh', 'lat', 'leav', 'lik', 'load', 'log', 'look', 'mak', 'many', 'me', 'mix', 'mod', 'mor

In [3]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-3-72b6ba6ac079>:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [4]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
modelo = Sequential()
modelo.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(len(train_y[0]), activation='softmax'))

In [5]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
modelo.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [6]:
# Fit the model
modelo.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
13/13 [==============================] - 1s 1ms/step - loss: 2.7421 - accuracy: 0.0627
Epoch 2/200
13/13 [==============================] - 0s 1ms/step - loss: 2.6882 - accuracy: 0.0764
Epoch 3/200
13/13 [==============================] - 0s 1ms/step - loss: 2.5404 - accuracy: 0.0452
Epoch 4/200
13/13 [==============================] - 0s 1ms/step - loss: 2.5400 - accuracy: 0.2623
Epoch 5/200
13/13 [==============================] - 0s 1ms/step - loss: 2.2656 - accuracy: 0.2672
Epoch 6/200
13/13 [==============================] - 0s 1ms/step - loss: 2.2712 - accuracy: 0.1881
Epoch 7/200
13/13 [==============================] - 0s 1ms/step - loss: 2.1704 - accuracy: 0.3247
Epoch 8/200
13/13 [==============================] - 0s 1ms/step - loss: 2.0292 - accuracy: 0.3464
Epoch 9/200
13/13 [==============================] - 0s 1ms/step - loss: 1.8743 - accuracy: 0.5132
Epoch 10/200
13/13 [==============================] - 0s 1ms/step - loss: 1.7641 - accuracy: 0.6318
Epoch 11/

In [7]:
modelo.save('saved_model/chatbot-brains-model')

INFO:tensorflow:Assets written to: saved_model/chatbot-brains-model\assets


In [8]:
import tensorflow as tf

loaded_model = tf.keras.models.load_model('saved_model/chatbot-brains-model')

In [23]:


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
    

def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    print("input_data: ",input_data)    
    
    results = loaded_model.predict([input_data])[0]
    print("results: ",results)
    
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [24]:
p = bow("HVAC is a mechanical system in a building I think", words)

print(p)
print(classes)

found in bag: hvac
found in bag: is
found in bag: a
found in bag: system
found in bag: in
found in bag: a
found in bag: build
found in bag: i
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
['check_ahu', 'check_all_hvac', 'check_boiler_plant', 'check_chiller_plant', 'check_energy', 'check_future_weather', 'check_how_it_should_all_work', 'check_power', 'check_runtime', 'check_zone_temps', 'goodbye', 'greeting', 'options', 'thanks', 'what_to_do']


In [25]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(loaded_model.predict(inputvar))

[[1.1322701e-05 9.9361801e-01 8.0619511e-06 8.2069064e-06 2.0572115e-07
  3.2828422e-04 8.5624952e-06 9.8246906e-05 1.6101414e-07 2.8381590e-05
  5.3282026e-08 5.8736335e-03 4.2884068e-07 2.3620801e-07 1.6230424e-05]]


In [26]:
classify_local('Hello, good day!')

found in bag: hello
found in bag: ,
found in bag: good
found in bag: day
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [1.0775472e-08 7.1436011e-07 1.8245773e-08 6.6444663e-09 2.4269323e-07
 4.6301629e-08 7.7450437e-09 2.9416057e-07 9.4652530e-10 3.9009430e-07
 4.6648879e-07 9.9999774e-01 5.1868105e-09 3.1666946e-08 1.2124188e-07]


[('greeting', '0.99999774')]

In [27]:
classify_local('How you can assist me?')

found in bag: how
found in bag: you
found in bag: can
found in bag: me
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  1.0  0.0  

[1 rows x 152 columns]
results:  [2.5807581e-06 1.1231040e-07 1.4518911e-06 1.6180411e-07 1.6185706e-05
 6.0758157e-07 2.8099514e-06 6.0054802e-08 3.3847434e-07 6.4008042e-08
 9.3690994e-07 2.6670832e-05 9.9991560e-01 9.4236593e-06 2.2989174e-05]


[('options', '0.9999156')]

In [28]:
classify_local('what is my mechanical system doing?')

found in bag: what
found in bag: is
found in bag: my
found in bag: system
found in bag: doing
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  1.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [3.8264729e-03 9.8003024e-01 4.0412619e-04 3.2501433e-05 2.9829444e-06
 6.4478409e-05 1.6475302e-04 6.5852637e-06 5.0503822e-06 3.5897447e-04
 2.8713421e-07 1.4949327e-02 8.0070844e-05 8.1764529e-06 6.6172419e-05]


[('check_all_hvac', '0.98003024')]

In [29]:
classify_local('are people complaining at all?')

found in bag: ar
found in bag: at
found in bag: al
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [4.1744574e-03 1.9441090e-03 2.0976881e-03 4.1539440e-01 3.0427601e-04
 1.6549496e-04 1.2665679e-02 2.6919506e-03 6.4560617e-03 3.6318070e-01
 1.1079893e-01 5.5706296e-02 3.5187168e-04 4.9486868e-03 1.9119311e-02]


[('check_chiller_plant', '0.4153944'), ('check_zone_temps', '0.3631807')]

In [30]:
classify_local('how is the HVAC running?')

found in bag: how
found in bag: is
found in bag: the
found in bag: hvac
found in bag: run
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [9.9185092e-07 9.9991345e-01 5.8244939e-07 1.6708578e-06 4.2436654e-09
 3.5500075e-06 5.6666663e-06 1.7665810e-06 8.9811202e-08 2.9558691e-06
 7.8484302e-11 6.8854897e-05 7.1808337e-08 5.7853352e-08 1.6813958e-07]


[('check_all_hvac', '0.99991345')]

In [31]:
classify_local('has the power use for the building been bad?')

found in bag: has
found in bag: the
found in bag: pow
found in bag: us
found in bag: for
found in bag: the
found in bag: build
found in bag: been
found in bag: bad
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [1.8020928e-06 5.8946935e-05 3.5096056e-04 1.0908003e-04 3.6781806e-02
 2.0429991e-04 8.0410209e-06 9.6141607e-01 5.3937780e-04 3.0330682e-06
 7.1463262e-05 2.9496191e-04 2.0768555e-05 2.4813125e-05 1.1455149e-04]


[('check_power', '0.96141607')]

In [32]:
classify_local('are we getting any free cooling or using mechanical cooling?')

found in bag: ar
found in bag: we
found in bag: get
found in bag: any
found in bag: fre
found in bag: cool
found in bag: or
found in bag: us
found in bag: cool
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [1.0914397e-01 4.8065741e-04 4.4232034e-03 4.1496554e-01 4.4249463e-01
 8.9456495e-03 4.7202708e-04 8.5561955e-04 1.5036102e-03 2.4181478e-04
 8.4031485e-03 1.2305197e-03 2.8124175e-03 3.9473297e-03 7.9984478e-05]


[('check_energy', '0.44249463'), ('check_chiller_plant', '0.41496554')]

In [33]:
classify_local('is the fan systems out on alarm?')

found in bag: is
found in bag: the
found in bag: fan
found in bag: system
found in bag: out
found in bag: on
found in bag: alarm
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [9.9999750e-01 1.1535463e-07 4.9209410e-07 8.4089962e-08 2.8225644e-09
 7.8655120e-08 1.7037112e-09 7.9080263e-11 1.6179213e-06 5.8911285e-09
 6.2496999e-09 1.2533968e-09 1.3659428e-08 2.5806333e-09 4.0836531e-10]


[('check_ahu', '0.9999975')]

In [34]:
classify_local('does that seem like its working correctly?')

found in bag: doe
found in bag: that
found in bag: lik
found in bag: work
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  1.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [0.0051697  0.00214835 0.00628475 0.00169718 0.00108877 0.00116341
 0.00099926 0.00591729 0.0669648  0.00168755 0.03483037 0.01561639
 0.00448001 0.09321579 0.75873643]


[('what_to_do', '0.75873643')]

In [35]:
classify_local('are any of the vav boxes not moving air?')

found in bag: ar
found in bag: any
found in bag: the
found in bag: vav
found in bag: box
found in bag: mov
found in bag: air
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [1.6416454e-03 2.1847896e-04 3.5950598e-06 2.0606874e-05 6.6135084e-09
 8.3595566e-09 5.9892125e-07 1.4724056e-06 1.2306629e-07 9.9810660e-01
 7.5768014e-09 6.5311224e-06 3.8097841e-08 1.2309047e-07 2.6349809e-07]


[('check_zone_temps', '0.9981066')]

In [36]:
classify_local('what are the discharge temps looking like on the reheat vav box coils?')

found in bag: what
found in bag: ar
found in bag: the
found in bag: discharg
found in bag: temp
found in bag: look
found in bag: lik
found in bag: on
found in bag: the
found in bag: reh
found in bag: vav
found in bag: box
input_data:         0    1    2    3    4    5    6    7    8    9    ...  142  143  144  \
input  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       145  146  147  148  149  150  151  
input  1.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 152 columns]
results:  [4.8167497e-04 9.0680876e-09 3.8637172e-08 5.1126417e-08 6.0157405e-14
 1.3107747e-13 4.0031539e-10 3.3269748e-11 1.6783028e-11 9.9951828e-01
 2.7741267e-13 1.8543107e-10 4.4385602e-12 5.3764129e-14 5.1636512e-12]


[('check_zone_temps', '0.9995183')]